# Gathering SQL Server indexes statistics and usage information

https://www.sqlshack.com/gathering-sql-server-indexes-statistics-and-usage-information/

In [ ]:
-- Identificando a fragmentação dos índices (Dirceu Resende) UNIQUE DATABASE
SELECT  DB_NAME()

SELECT  OBJECT_NAME(B.object_id) AS TableName, B.name AS IndexName, A.index_type_desc AS IndexType, A.avg_fragmentation_in_percent
FROM    sys.dm_db_index_physical_stats(DB_ID(), NULL, NULL, NULL, 'LIMITED') A
INNER   JOIN sys.indexes B WITH(NOLOCK) ON B.object_id = A.object_id AND B.index_id = A.index_id
WHERE   A.avg_fragmentation_in_percent > 30
    AND OBJECT_NAME(B.object_id) NOT LIKE '[_]%'
    AND A.index_type_desc != 'HEAP'
ORDER BY    A.avg_fragmentation_in_percent DESC

In [ ]:
-- Rebuid OR Reorganize Index
use dbCrmActivesoft

SELECT DB_NAME()

DECLARE @Database NVARCHAR(255)   
DECLARE @Table NVARCHAR(255)
DECLARE @Index NVARCHAR(255)
DECLARE @Fragmentation FLOAT
DECLARE @cmd NVARCHAR(1000)  


DECLARE TableCursor CURSOR READ_ONLY FOR 
    SELECT DB_NAME(), OBJECT_NAME(ind.OBJECT_ID), ind.name, indexstats.avg_fragmentation_in_percent
    FROM sys.dm_db_index_physical_stats(DB_ID(), NULL, NULL, NULL, NULL) indexstats 
    INNER JOIN sys.indexes ind ON ind.object_id = indexstats.object_id  AND ind.index_id = indexstats.index_id 
    WHERE indexstats.avg_fragmentation_in_percent > 5    AND ind.Name is not null 
    ORDER BY indexstats.avg_fragmentation_in_percent DESC
    
    OPEN TableCursor   

   FETCH NEXT FROM TableCursor INTO @Database, @Table, @Index, @Fragmentation
   WHILE @@FETCH_STATUS = 0   
   BEGIN
      BEGIN TRY
        IF @Fragmentation > 30
            SET @cmd = 'ALTER INDEX [' + @Index + '] ON ' + @Database + '.dbo.' +@Table + ' REBUILD' 
        ELSE
            SET @cmd = 'ALTER INDEX [' + @Index + '] ON ' + @Database + '.dbo.' +@Table + ' REORGANIZE' 

          PRINT @cmd -- uncomment if you want to see commands
        --  EXEC (@cmd) 
      END TRY
      BEGIN CATCH
         PRINT '---'
         PRINT @cmd
         PRINT ERROR_MESSAGE() 
         PRINT '---'
      END CATCH

      FETCH NEXT FROM TableCursor INTO @Database, @Table, @Index, @Fragmentation
   END   

   CLOSE TableCursor   
   DEALLOCATE TableCursor

## Missing Indexes in SQL Server

https://dbtut.com/index.php/2019/04/21/how-to-find-missing-indexes-in-sql-server/

In [ ]:
-- Sugerido por Joao

CREATE INDEX [IX_TbCaixaLancamento_CdCaixaTipoLancamento_CdTipoRecebimento] ON [dbo].[TbCaixaLancamento] ([CdCaixaTipoLancamento],[CdTipoRecebimento]) INCLUDE ([IdCaixaMovimentacao],[Valor])

    -- Igual ao Sugerido pelas Estatísticas do Banco
    CREATE INDEX [IX_TbCaixaMovimentacao_IdCaixaAbertura] ON [dbo].[TbCaixaMovimentacao] ([IdCaixaAbertura])



-- Sugerido pelas Estatísticas do Banco

CREATE INDEX [IX_TbCaixaLancamento_IdCaixaMovimentacao_CdTipoRecebimento] ON [dbSigaGGE].[dbo].[TbCaixaLancamento] ([IdCaixaMovimentacao], [CdTipoRecebimento]) INCLUDE ([Valor])

    -- Igual ao solicitado por joão
    CREATE INDEX [IX_TbCaixaMovimentacao_IdCaixaAbertura] ON [dbSigaGGE].[dbo].[TbCaixaMovimentacao] ([IdCaixaAbertura])

## Raio-X - Filtered index

SQL Server Filtered Indexes What They Are, How to Use and Performance Advantages
    https://www.mssqltips.com/sqlservertip/1785/sql-server-filtered-indexes-what-they-are-how-to-use-and-performance-advantages/

What You Can (and Can’t) Do With Filtered Indexes
    https://www.brentozar.com/archive/2013/11/what-you-can-and-cant-do-with-filtered-indexes/

Filtered Indexes and Dynamic SQL
    https://www.brentozar.com/archive/2013/11/filtered-indexes-and-dynamic-sql/

## Dynamic Management Views (DMVs) To List Foreign Keys With No Index

## Should every SQL Server foreign key have a matching index?
    https://stackoverflow.com/questions/3650690/should-every-sql-server-foreign-key-have-a-matching-index

### SQL Server Best Practices: Using Filegroups to Implement Cold and Hot Data Isolation Backup Solution

    https://www.alibabacloud.com/blog/sql-server-best-practices-using-filegroups-to-implement-cold-and-hot-data-isolation-backup-solution_594480